In [12]:
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
import pandas as pd
import re
from sklearn.svm import LinearSVC
import nltk as nl



data = pd.read_csv("/home/muskan_2117/DAIICT/Sem2/IT550/InformationRetrieval/Tweets.csv")

y = data.airline_sentiment
x = data.text



word_vector = []
temp = []

for text in x:
    r = re.sub(r'[^\w\s-]','',text)
    r = re.sub(r'[0-9]','',r)
    r = r.lower()
    word_vector.append(r)
    
word_vector = [nl.word_tokenize(sentences) for sentences in word_vector]


model = Word2Vec(word_vector, size=100, window = 3, min_count=1, workers=4, sg =1)

model.train(word_vector, total_examples=14640, epochs=10)


#vectors = model[model.wv.vocab]
#words = list(model.wv.vocab)


#print(model)
#print(vectors)




(1908518, 2519480)

In [13]:
w2v = dict(zip(model.wv.index2word, model.wv.vectors))

In [14]:
import numpy as np

class MeanEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        # if a text is empty we should return a vector of zeros
        # with the same dimensionality as all the other vectors
        self.dim = len(next(iter(word2vec.items())))

    def fit(self, X, y):
        return self

    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec]
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])


In [15]:
features = MeanEmbeddingVectorizer(w2v)

x = features.transform(word_vector)

In [16]:
x_train, x_test, y_train, y_test = train_test_split(x, y,test_size=0.2)

In [17]:
clf = LinearSVC()

clf.fit(x_train,y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [18]:
pred_output = clf.predict(x_test)

In [20]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
print("SVM F1 Score -> ",f1_score(pred_output, y_test, average=None))
print("SVM Recall Score -> ",recall_score(pred_output, y_test, average=None))
print("SVM Precision Score -> ",precision_score(pred_output, y_test, average=None))


SVM F1 Score ->  [0.86601553 0.56538462 0.63183475]
SVM Recall Score ->  [0.80269266 0.67898383 0.76246334]
SVM Precision Score ->  [0.94018488 0.48434926 0.53941909]
